In [1]:
import numpy as np
import cv2
import glob

In [2]:
key = cv2. waitKey(1)
webcam = cv2.VideoCapture(0)
ctr = 0;

while True:
    check, frame = webcam.read()
    cv2.imshow("Capturing", frame)
    key = cv2.waitKey(1)
    if key == ord('s'): 
        cv2.imwrite(filename='images/cam/saved_img-'+str(ctr)+'.jpg', img=frame)
        ctr = ctr + 1
    elif key == ord('q'):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break


Turning off camera.
Camera off.
Program ended.


In [13]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((5*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)*36

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('images/cam/saved_img-*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,5),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,5), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(0)

cv2.destroyAllWindows()

In [7]:
#Camera matrix, distortion coefficients, rotation and translation vectors...
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [10]:
dist

array([[-5.64484341e+00,  5.61585671e+02, -1.51098451e-01,
         6.90227001e-03, -9.21067765e+03]])

In [11]:
from tabulate import tabulate
print(tabulate(rvecs))

----------  ----------  -------
-0.1508     -0.0487766  1.57856
-0.148221   -0.155977   1.56851
-0.0494512  -0.160741   1.87156
-0.239504   -0.0301164  1.2363
----------  ----------  -------


In [12]:
print(tabulate(tvecs))

-------  --------  -------
265.127  -155.232  3617.31
145.163  -157.549  3664.62
188.013  -169.993  3731.85
163.873  -216.887  3721.12
-------  --------  -------


In [17]:
#Undistortion
img = cv2.imread('images/cam/saved_img-3.jpg')
#cv2.imshow('img',img)
#cv2.waitKey(500)
h, w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [18]:
## First method (Using cv2.undistort())

# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)


# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)


True